In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# client_key, client_secret, sierra_username, sierra_password
import patron_batch_upload_vars as vars

connection_uri = 'postgres://{}:{}@sierra-db.plch.net:1032/iii'.format(vars.sierra_username, vars.sierra_password)

# use sqlalchemy to create our database connection
engine = create_engine(connection_uri)

In [2]:
# grab some patron data to create a test spreadsheet ...
sql = '''
SELECT
r.record_num as patron_record_num,
r.id as patron_record_id,
p.birth_date_gmt, 
n.first_name,
n.middle_name,
n.last_name

FROM
sierra_view.record_metadata as r

JOIN
sierra_view.patron_record as p
ON
  p.record_id = r.id

JOIN
sierra_view.patron_record_fullname as n
ON
  n.patron_record_id = p.record_id

WHERE
r.record_type_code = 'p'
AND r.campus_code = ''
AND p.birth_date_gmt IS NOT NULL

limit 1000

'''

df = pd.read_sql(sql=sql, con=engine)

df.shape

(1000, 6)

In [3]:
# save the dataframe to an excel spreadsheet ...
df.to_excel('test_output.xlsx', index=False)

In [4]:
# clear the dataframe, and reload it from the test excel sheet
df = None

df = pd.read_excel('test_output.xlsx')
df.shape

(1000, 6)

In [5]:
# this is the sql we're going to iterate on ...
sql = '''
SELECT
string_agg(r.record_num::TEXT, ', ') as patron_record_nums

FROM
sierra_view.patron_record_fullname as n

JOIN
sierra_view.patron_record as p
ON
  p.record_id = n.patron_record_id

JOIN
sierra_view.record_metadata as r
ON
  r.id = n.patron_record_id


WHERE
n.last_name = %s
AND n.first_name = %s
'''

with engine.connect() as con:
    for i, row in df.iterrows():
        
        print('count: {}'.format(i,), end='\n')
        
        # run the query, get the row (result) ... 
        result = con.execute(sql, [ row['last_name'], row['first_name'] ]).fetchone()
        # -- do something with the results ...
        print('possible matching patron records: {}'.format(result), end='\n----\n')
        
        # maybe just run the first 10 as a test ...
        if (i >= 10):
            break

count: 0
possible matching patron records: ('2434265, 1427238',)
----
count: 1
possible matching patron records: ('1277517, 1260612, 1436396, 2161540',)
----
count: 2
possible matching patron records: ('1956183',)
----
count: 3
possible matching patron records: ('2445807',)
----
count: 4
possible matching patron records: ('2436702',)
----
count: 5
possible matching patron records: ('1951979',)
----
count: 6
possible matching patron records: ('2139969',)
----
count: 7
possible matching patron records: ('2309517, 2433173, 2433189, 2433202, 2433226, 2433243, 2433181, 2433178, 2433180, 2433188, 2433174, 2433172, 2433177, 2433193, 2433183, 2433194, 24332 ... (384 characters truncated) ... 33191, 2433241, 2433246, 2433190, 2433199, 2433209, 2433220, 2433196, 2433216, 2433217, 2433239, 2433245, 2433213, 2433221, 2433222, 2433232, 2433236',)
----
count: 8
possible matching patron records: ('1820953, 2092901, 1401214',)
----
count: 9
possible matching patron records: ('1940405',)
----
count: 10